# Investigate HIV Datasets

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The human immunodeficiency virus (HIV) is a lentivirus (a subgroup of retrovirus) that causes HIV infection and over time acquired immunodeficiency syndrome (AIDS).AIDS is a condition in humans in which progressive failure of the immune system allows life-threatening opportunistic infections and cancers to thrive. Without treatment, average survival time after infection with HIV is estimated to be 9 to 11 years, depending on the HIV subtype([Source](https://en.wikipedia.org/wiki/HIV)). It is one of the most dangerous virus that is threatening human beings' lives, not only adults but also children. 

In this report I will investigate a few HIV datasets which are downloaded from [Gapmind](http://www.gapminder.org/data/). The datasets are listed below:

* **Child-HIV .csv:** Child death rate in year 2008 in different countries, per thousand
* **1-59-HIV .csv:** Total number of child death in year 2008 in different countries
* **indicator_hiv_plwha.csv:** Total number of HIV livings in different years and countries 
* **Indicator_Estimated new HIV cases All ages.csv:** Total number of people among all age groups that were newly infected with HIV during different years
* **Indicator_HIV Incidence Ages 15-49.csv:** Number of people between 15 and 49 years old that were newly infected with HIV during the given year per 100 population
* **Indicator_Annual number of AIDS deaths.csv:** Total number of deaths due to AIDS during different years
* **indicator hiv estimated prevalence% 15-49.csv:** Number of people living with HIV per 100 population of age group 15-49


I would also feel interested in continent-wise distribution of different variables, so I need an additional data source to translate from countries to continents. I am using [this dataset](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv).

I am hoping to answer a few questions from data in hand: 

* Which country is the most dangerous in terms of the number of people infected and death percentage. 
* What's the trends over time of this top country.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

Information is stored in different datasets, so first I need to combine them and wrangle for further exploration. 

### General Properties

In [ ]:
# Load different datasets and combine and wrangle data for further investigation
child_death_per_k = pd.read_csv('dataset/Child-HIV .csv')
child_death_ttl = pd.read_csv('dataset/1-59-HIV .csv') 
conti_country = pd.read_csv('dataset/all.csv')
conti_country = conti_country.loc[:, ["name", "region"]]
conti_country.columns = ["Country", "Continent"]

child_death = pd.merge(child_death_ttl, child_death_per_k, how="left", on = "Gapminder name")
child_death.columns = ["Country", "Total", "Num_per_thousand"]
child_death = pd.merge(child_death, conti_country, how="left", on = "Country")

child_death.head()

In [ ]:
child_death.info()

In [ ]:
child_death.describe()

The new dataset `child_death` combines `Child-HIV .csv` and `1-59-HIV .csv`, which demonstrates total number of death and number per thousand in year 2008.

From above we can get some initial information:

* There are 178 observations in the dataset
* The average total number of death is 1130 but median is 6 (1.72 and 0.05 respectively for number per thousand), which means these two variables are highly skewed
* There is a few missing data under continent column, which will be further discussed in the following steps

In [ ]:
def melt_func(x):
    """
    Melt dataframe with similar data structure
    """
    idx = x.columns[0]
    return x.melt(id_vars = idx)

In [ ]:
def outer_merge(x, y):
    """
    Outer merge two dataframes with keys of country and year
    """
    return pd.merge(x, y, how="outer", on=["Country", "Year"])

In [ ]:
ttl_living_HIV = pd.read_csv('dataset/indicator_hiv_plwha.csv')
ttl_living_HIV = melt_func(ttl_living_HIV)
ttl_living_HIV.columns = ['Country', 'Year', 'Num_HIV_living']

new_HIV_all_age = pd.read_csv('dataset/Indicator_Estimated new HIV cases All ages.csv')
new_HIV_all_age = melt_func(new_HIV_all_age)
new_HIV_all_age.columns = ['Country', 'Year', 'New_added_HIV_all_age']

new_HIV_15_49_pct = pd.read_csv('dataset/Indicator_HIV Incidence Ages 15-49.csv')
new_HIV_15_49_pct = melt_func(new_HIV_15_49_pct)
new_HIV_15_49_pct.columns = ['Country', 'Year', 'New_added_HIV_15-49_pct']

ann_death = pd.read_csv('dataset/Indicator_Annual number of AIDS deaths.csv')
ann_death = melt_func(ann_death)
ann_death.columns = ['Country', 'Year', 'Annual_death']

est_pct_15_49 = pd.read_csv('dataset/indicator hiv estimated prevalence% 15-49.csv')
est_pct_15_49 = melt_func(est_pct_15_49)
est_pct_15_49.columns = ['Country', 'Year', 'Est_HIV_pct_15-49']

ttl_year_data = outer_merge(ttl_living_HIV, 
                    outer_merge(new_HIV_all_age,
                    outer_merge(new_HIV_15_49_pct, 
                    outer_merge(est_pct_15_49, ann_death))))

ttl_year_data = pd.merge(ttl_year_data, conti_country, how="left", on="Country")

ttl_year_data.head(10)

In [ ]:
ttl_year_data.info()

In [ ]:
ttl_year_data.describe()

The new dataset `ttl_year_data` combines all the rest datsets. Every dataset needs to be unpivoted first and then joined according to column `country` and `year`. 

From above we can also get some initial insights:

* There are total 9097 observations and 5 columns
* There are many missing data, including continents
* The largest number of HIV living is 5.6 * 10^6, which is extremely huge. The mean is 1.57 * 10^5 and the median is 1.2 * 10^4
* The largest number of new added HIV of all ages is 7.2 * 10^5. The mean is 3.5 * 10^4 and the median is 6.2 * 10^3.
* The mean of numbers of HIV newly infected from 15 to 49 is 0.46 and median is 0.42. This is pretty close. 
* The mean of numbers of HIV infected cases per hundred is 1.74 and median is 0.3. 
* The mean of annual death is 9050 and median is 350, which is highly skewed. 


### Data Cleaning

As previous initial exploration, we can see there are two kinds of missing data: 

* numbers under different indicator columns
* strings under continent column

Let's discuss them separately.

#### Missing data under different indicator columns

In [ ]:
cleaned_year_data = ttl_year_data.copy()
cleaned_year_data.groupby('Year').describe()

As I find previously there are a lot of missing data, probably due to limited resources or initiative in early years. Some common approaches to fill missing data are:

* filled with mean or median
* look for new data source to fill the gap
* discard observations with missing data

Since values of different indicator are quite different among different years and countries, it is not accurate to simply use mean or median to fill at a specific year or country, so it wouldn't be a good idea to go with option one. I don't think we can find a new data source either, or at least at low cost. So for simplicity, I will just discard these years with a lot of missing data (years before 1990) and remain the rest. 

In [ ]:
cleaned_year_data = cleaned_year_data.loc[cleaned_year_data.Year >= '1990',]

#### Missing data under continent column

Missing data under continent column is kind of easier to deal with. First I will query out those missing continent in two combined datasets, and then fill them in one by one.

In [ ]:
# query countries with missing continent info in child_death dataset

missing_conti = list(child_death.loc[pd.isnull(child_death.Continent), :].Country)
missing_conti

In [ ]:
cleaned_year_data.loc[pd.isnull(cleaned_year_data.Continent), :].Country

It's surprising to see some of countries' name are missing, so it's impossible to know which continent they belong. These observations will also be deleted. 

In [ ]:
# Remove all data w/o country info
cleaned_year_data = cleaned_year_data.loc[-pd.isnull(cleaned_year_data.Country), :]

# Query countries with missing conti info and combine with the previous one
missing_conti += list(cleaned_year_data.loc[pd.isnull(cleaned_year_data.Continent), :].Country)
missing_conti_set = set(missing_conti)
missing_conti_set

Of all these countries, they may have been reconstructed or reorganized. But in order to conduct further analysis, I will mannually set up a dictionary to map these countries to continents.

In [ ]:
missing_conti_map = {
 'Abkhazia': 'Asia',
 'Akrotiri and Dhekelia': 'Europe',
 'Antarctica': 'Antarctica',
 'Bolivia': 'America',
 'Bonaire': 'America',
 'British Virgin Islands': 'America',
 'Brunei': 'Asia',
 'Cape Verde': 'Africa',
 'Channel Islands': 'Europe',
 'Chinese Taipei': 'Asia',
 'Christian': 'Africa',
 'Christmas Island': 'Asia',
 'Coastline': 'Nan',
 'Cocos Island': 'Asia',
 'Congo, Dem. Rep.': 'Africa',
 'Congo, Rep.': 'Africa',
 'Cook Is': 'Nan',
 "Cote d'Ivoire": 'Africa',
 'Cura\x8dao': 'Nan',
 'Czechoslovakia': 'Europe',
 'East Germany': 'Europe',
 'Eritrea and Ethiopia': 'Africa',
 'Faeroe Islands': 'Europe',
 'Falkland Is (Malvinas)': 'America',
 'Hawaiian Trade Zone': 'America',
 'Hong Kong, China': 'Asia',
 'Iran': 'Asia',
 'Kosovo': 'Europe',
 'Kyrgyz Republic': 'Asia',
 'Lao': 'Asia',
 'Macao, China': 'Asia',
 'Macedonia, FYR': 'Europe',
 'Micronesia, Fed. Sts.': 'Oceania',
 'Moldova': 'Europe',
 'Netherlands Antilles': 'Europe',
 'Ngorno-Karabakh': 'Europe',
 'North Korea': 'Asia',
 'North Yemen (former)': 'Asia',
 'Northern Cyprus': 'Europe',
 'Reunion': 'Africa',
 'Russia': 'Europe',
 'Saba': 'Europe',
 'Saint Eustatius': 'Europe',
 'Sark': 'Europe',
 'Serbia and Montenegro': 'Europe',
 'Serbia excluding Kosovo': 'Europe',
 'Slovak Republic': 'Europe',
 'Somaliland': 'Africa',
 'South Korea': 'Asia',
 'South Ossetia': 'Europe',
 'South Yemen (former)': 'Asia',
 'St. BarthŽlemy': 'Nan',
 'St. Helena': 'Africa',
 'St. Kitts and Nevis': 'America',
 'St. Lucia': 'America',
 'St. Martin': 'America',
 'St. Martin (French part)': 'Europe',
 'St. Vincent and the Grenadines': 'America',
 'St.-Pierre-et-Miquelon': 'America',
 'Svalbard': 'Europe',
 'Syria': 'Asia',
 'Taiwan': 'Asia',
 'Tanzania': 'Africa',
 'Transnistria': 'Europe',
 'U.S. Pacific Islands': 'America',
 'USSR': 'Asia',
 'United Kingdom': 'Europe',
 'United Korea (former)\r\n': 'Asia',
 'United States': 'America',
 'Venezuela': 'America',
 'Vietnam': 'Asia',
 'Virgin Islands, British': 'America',
 'Wake Island': 'Oceania',
 'Wallis et Futuna': 'Oceania',
 'West Bank and Gaza': 'Europe',
 'West Germany': 'Europe',
 'Yugoslavia': 'Europe',
 '\x81land': 'Nan', 
 'Central African Rep.': 'Africa',
 'Czech Rep.': 'Europe',
 'Korea, Dem. Rep.': 'Asia',
 'Dominican Rep.': 'America',
 'Laos': 'Asia',
 'Korea, Rep.': 'Asia',
 'Yemen, Rep.': 'Asia'}

# Fill NAs with above dictionary
cleaned_year_data.Continent = cleaned_year_data.Continent.fillna(missing_conti_map)
child_death.Continent = child_death.Continent.fillna(missing_conti_map)

<a id='eda'></a>
## Exploratory Data Analysis



### Which country is the most dangerous in terms of the number of people died and death rate?

In [ ]:
child_death_top10_1 = child_death.sort_values(['Total'], ascending = False).iloc[0:10]
child_death_top10_1.plot('Country', 'Total', kind='barh').invert_yaxis()
plt.title('Bar Plot of Top 10 Total Num of Child Death Countries in 2008')
plt.show()

In [ ]:
child_death_top10_1.Continent.value_counts().plot(kind = 'pie')
plt.show()

In [ ]:
## Percentage of these top 10 countries
child_death_top10_1.Total.sum() / child_death.Total.sum()

At first I take a look at child death in 2008. It is surprising to see there are 9 countries in Africa among top 10. These are South Africa, Nigeria, Mozambique, Tanzania, Uganda, Zambia, Ethiopia, Kenya and Malawi. The only Asian country is India. It is also surprising to see the total number of death among top 10 countries accounts for 73% of total global death.

I then replot the bars by different continents. Not surprisingly Africa counts for tha majority of death (92%).

In [ ]:
## Total death among all countries

child_death.groupby(['Continent']).Total.sum().reset_index().sort_values('Total', ascending = False).plot("Continent", "Total", kind='bar')

In [ ]:
# Calculate percentage Africa accounts for the total
child_death.loc[child_death.Continent == "Africa", ].Total.sum()/child_death.Total.sum()


Let's then take a look at permill(number per thousand) child death in 2008.

In [ ]:
data = {'from_year': [2010, 2011, 2012, 2011, 2012, 2010, 2011, 2012],
    'name': ['John', 'John1', 'John', 'John', 'John4', 'John', 'John1', 'John6'],
    'out_days': [11, 8, 10, 15, 11, 6, 10, 4]}
persons = pd.DataFrame(data, columns=["from_year", "name", "out_days"])
days_off_yearly = persons.groupby(["from_year", "name"])['out_days'].sum()
print(days_off_yearly)

In [ ]:
print (days_off_yearly.reset_index()
                      .sort_values(['from_year','out_days'],ascending=False))

In [ ]:
child_death_top10_2 = child_death.sort_values('Num_per_thousand', ascending=False).iloc[:10]
child_death_top10_2.plot('Country', 'Num_per_thousand', kind = 'barh').invert_yaxis()
plt.title('Bar Plot of Top 10 Permill of Child Death Countries in 2008')
plt.show()

It seems that Swaziland surpasses South Africa and becomes the top one Permill of Child Death this time. It is understandable since Swaziland has much less population: 1.3 million for Swaziland VS. 55 million for South Africa. If we consider the total impact, South Africa definitely is our target.

In [ ]:
child_death_top10_2.Continent.value_counts().plot(kind = 'pie')
plt.show()

All top 10 countries are in Africa. India drops out this time, one possible guess is that India has a much larger total population basis which makes the permill very small. 

Throughout this quick review we can find HIV is a public concern and cause of death in many parts of Africa. There are more than 200k children died from HIV just in 2008, which is an astonishing number. A much more astonishing fact is that 92% happens in Africa! A quick review on [HIV/AIDS in Africa of wikipedia](https://en.wikipedia.org/wiki/HIV/AIDS_in_Africa) would also confirm my findings.

> HIV/AIDS is a major public health concern and cause of death in many parts of Africa. Although the continent is home to about 15.2 percent of the world's population, more than two-thirds of the total, some 35 million infected, were Africans, of whom 15 million have already died in Sub-Saharan Africa alone accounted for an estimated 69 percent of all people living with HIV and 70 percent of all AIDS deaths in 2011. In the countries of sub-Saharan Africa most affected, AIDS has raised death rates and lowered life expectancy among adults between the ages of 20 and 49 by about twenty years. Furthermore, the life expectancy in many parts of Africa is declining, largely as a result of the HIV/AIDS epidemic with life-expectancy in some countries reaching as low as thirty-four years.



In [ ]:
ttl_death_all_year = cleaned_year_data.groupby(['Country']).Annual_death.sum().reset_index()
ttl_death_all_year = ttl_death_all_year.loc[-ttl_death_all_year.Annual_death.isnull(),:].sort_values("Annual_death", ascending = False)
top10_ttl_death_all_year = ttl_death_all_year.head(10)
top10_ttl_death_all_year.plot('Country', 'Annual_death', kind='bar')

I also plot a bar graph indicating total number of death over all years after 1990. It is easy to see most of them are in Africa and South Africa still has the largest death.

From above exploration, I feel confident to answer my first question: 

**Which country is the most dangerous in terms of the number of people infected and death percentage?**

The answer is **South Africa**. 


______



### What is the trends?

In the following exploration, I will try to understand the trends in South Africa. How does the number of HIV living people change over time, up or down? Is there an effective approach to stop newly added ones? How is the percentage goes?

Let's start!

In [ ]:
south_africa = cleaned_year_data.loc[cleaned_year_data.Country == 'South Africa']
south_africa

In [ ]:
# Explore number of total and newly added HIV living

south_africa.plot('Year', 'Num_HIV_living', kind='line')
south_africa.plot('Year', 'New_added_HIV_all_age', kind='line')

From two above plots we can see the new added number of HIV livings grows rapidly at first, which accounts for the sharp growth of total number of HIV livings. If we can add a tangent line in the upper graph, we can find the slope is growing. 

The turning point happens at around 1998. The new added HIV livings derease rapidly, which can be considered as a result of government and other organizations. Although the total number of HIV livings is still growing, the speed of growth is much slower.

In [ ]:
south_africa.plot("Year", "New_added_HIV_15-49_pct", kind="line")

This plot of percentage also shows similar trends. The turning point is around 1998. The interesting part is the percentage decrease a little bit more than previous absolute number. This may be due to the growth of population basis.

If we take a look at HIV percentage from 15-49, we can also find a point around 1998 that the rate or slope of growth is decreasing and relatively stable in the following years. 

In [ ]:
south_africa.plot("Year", "Est_HIV_pct_15-49", kind="line")

In the end we take a look at the graph of annual death. Its trend has a few years delay than prevoius plots. It is understandable since it will take some years before affected HIV livings die. The interesting part is the annual death decreases after 2007.

In [ ]:
south_africa.plot("Year", "Annual_death", kind="line")

<a id='conclusions'></a>
## Conclusions

Throughout the above exploration, we 

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!